In [1]:
import sqlite3

try:
    conn = sqlite3.connect('data.db')
    cursor = conn.cursor()

    # Execute a SELECT query
    cursor.execute('SELECT * FROM items')

    # Fetch all rows
    rows = cursor.fetchall()

    # Print the retrieved data
    for row in rows:
        print(f"ID: {row[0]}, Name: {row[1]}, Price: {row[2]}, Location: {row[3]}")

    # Close the connection
    conn.close()

except Exception as e:
    print(f"Error: {str(e)}")

ID: 1, Name: Pizza Kebap Haus, Price: 8.0, Location: {"latitude":49.1448876875684,"longitude":9.220654070377352}
ID: 2, Name: Divan 2, Price: 7.5, Location: {"latitude":49.14463853901513,"longitude":9.220721125602724}
ID: 3, Name: Döner Eck, Price: 7.0, Location: {"latitude":49.14468591242803,"longitude":9.22106981277466}
ID: 4, Name: MEZOPOTAMYA, Price: 5.5, Location: {"latitude":49.14551055347107,"longitude":9.220029115676882}
ID: 5, Name: Star Kebap, Price: 6.0, Location: {"latitude":49.143134848264644,"longitude":9.22031342983246}
ID: 6, Name: Merkez, Price: 6.5, Location: {"latitude":49.14192414227402,"longitude":9.218368828296663}
ID: 7, Name: Stern Kebap, Price: 8.0, Location: {"latitude":49.14055372777727,"longitude":9.22032952308655}
ID: 8, Name: Otantik, Price: 7.0, Location: {"latitude":49.13836203863693,"longitude":9.220359027385713}
ID: 9, Name: Alaturka, Price: 10.0, Location: {"latitude":49.14159601830976,"longitude":9.217143058776857}
ID: 10, Name: Aleppo, Price: 7.0, L

In [18]:
import folium
from folium.plugins import HeatMap
import sqlite3
import json

def create_heatmap():
    try:
        # Connect to SQLite database
        conn = sqlite3.connect('data.db')
        cursor = conn.cursor()

        # Execute a SELECT query
        cursor.execute('SELECT * FROM items')

        # Fetch all rows
        rows = cursor.fetchall()

        # Create a Folium map centered on the first data point
        map_center = json.loads(rows[0][3])  # Parse the JSON string
        my_map = folium.Map(location=[map_center['latitude'], map_center['longitude']], zoom_start=14)

        # Create a list of data points for the heatmap
        heat_data = []
        for row in rows:
            location = json.loads(row[3])  # Parse the JSON string
            heat_data.append([location['latitude'], location['longitude'], row[2]])  # [latitude, longitude, price]

            # Determine the min and max price values
            min_price = min(heat[2] for heat in heat_data)
            max_price = max(heat[2] for heat in heat_data)

            # Add markers for each location
            folium.Marker(
                location=[location['latitude'], location['longitude']],
                popup=f"Name: {row[1]}<br>Price: {row[2]}",
                icon=folium.Icon(color='blue')
            ).add_to(my_map)

        # Create a gradient for the heatmap based on price
        gradient = {min_price: 'blue', max_price: 'red'}

        # Create a heatmap with colors based on price
        HeatMap(heat_data, radius=15, gradient=gradient, opacity=0.5, blur=25, max_zoom=14).add_to(my_map)

        # Create a heatmap with all data points
        #HeatMap(heat_data, radius=30, gradient={gradient},
                #min_opacity=0.4, max_opacity=0.7, blur=40, max_zoom=14).add_to(my_map)

        # Save the map as an HTML file
        my_map.save('heatmap.html')

        # Close the connection
        conn.close()

    except Exception as e:
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    create_heatmap()


In [24]:
import folium
from folium.plugins import HeatMap
import sqlite3
import json

from IPython.display import display

def normalize(value, min_val, max_val):
    # Normalize value to [0, 1]
    return (value - min_val) / (max_val - min_val)

def create_price_heatmap():
    try:
        # Connect to SQLite database
        conn = sqlite3.connect('data.db')
        cursor = conn.cursor()

        # Execute a SELECT query
        cursor.execute('SELECT * FROM items')

        # Fetch all rows
        rows = cursor.fetchall()

        # Create a Folium map centered on the first data point
        map_center = json.loads(rows[0][3])  # Parse the JSON string
        my_map = folium.Map(location=[map_center['latitude'], map_center['longitude']], zoom_start=13)

        # Create a list of data points for the heatmap
        heat_data = []
        prices = [row[2] for row in rows]  # Extract prices from rows

        # Normalize price values to [0, 1]
        min_price = min(prices)
        max_price = max(prices)
        normalized_prices = [normalize(price, min_price, max_price) for price in prices]

        for i, row in enumerate(rows):
            location = json.loads(row[3])  # Parse the JSON string
            heat_data.append([location['latitude'], location['longitude'], normalized_prices[i]])  # [latitude, longitude, normalized_price]

            # Add markers for each location
            folium.Marker(
                location=[location['latitude'], location['longitude']],
                popup=f"Name: {row[1]}<br>Price: {row[2]}",
                icon=folium.Icon(color='blue')
            ).add_to(my_map)

        # Create a gradient for the heatmap based on normalized prices
        gradient = {0: 'blue', 0.5: 'green', 1: 'red'}

        # Create a heatmap with colors based on normalized prices
        HeatMap(heat_data, radius=25, gradient=gradient, opacity=0.5, blur=15, max_zoom=14).add_to(my_map)

        # Save the map as an HTML file
        my_map.save('price_heatmap.html')
        display(my_map)

        # Close the connection
        conn.close()

    except Exception as e:
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    create_price_heatmap()


Error: name 'my_Map' is not defined
